<a href="https://colab.research.google.com/github/andrerizzo/Used_Cars_Price_Prediction/blob/master/Used_Cars_Price_Prediction_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previsão de preços de carros usados

### Etapa 1: Entendimento do problema

#### Introdução  
<br>

#### Objetivo  
<br>

#### Base de Dados - Atributos

* **Age**:&nbsp; Idade do paciente  *\[anos]*  

---  

<br>  

### Etapa 2: Entendimento dos dados

#### Instalar pacotes

In [1]:
!pip install joblib
!pip install lazypredict

#### Carregar bibliotecas

In [13]:
# Bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
from scipy.stats import skew, kurtosis
import math

# Biblioteca para salvar modelo
import joblib

# Biblioteca para descompactar arquivo
import zipfile
import requests

# Biblioteca para análise dos melhores modelos
from lazypredict.Supervised import LazyClassifier

# Scikit Learn Misc
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif, RFE, RFECV
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Modelos
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

# Hyperparameters Optimization
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report


<br>

#### Carregar base de dados

#### Download do Kaggle

In [18]:
!wget https://github.com/andrerizzo/Used_Cars_Price_Prediction/raw/refs/heads/master/Dataset/train.zip

--2024-09-23 01:31:14--  https://github.com/andrerizzo/Used_Cars_Price_Prediction/raw/refs/heads/master/Dataset/train.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/andrerizzo/Used_Cars_Price_Prediction/refs/heads/master/Dataset/train.zip [following]
--2024-09-23 01:31:15--  https://raw.githubusercontent.com/andrerizzo/Used_Cars_Price_Prediction/refs/heads/master/Dataset/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4714876 (4.5M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]   4.50M  --.-KB/s    in 0.07s   

2024-09-23 01:31:15 (65.9 M

#### Extract file from zip

In [19]:
zip = zipfile.ZipFile('train.zip', 'r')
zip.extractall()

#### Criar dataframe

In [20]:
df = pd.read_csv('train.csv')

In [21]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


#### Verificação de atributos nulos

In [24]:
nf = df.isnull().any().sum()
print('Número de observações nulas:', nf)

Número de observações nulas: 3


In [27]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


Observações:
* Os veículos que não apresentam tipo de combustível são elétricos.  
* Para os veículos que apresentam status de acidente como *NONE REPORTED*, é possível substituir os valores em branco do atributo *CLEAN_TITLE* &nbsp;como **YES**.
* Para os veículos que apresentam status de acidente diferente de *NONE REPORTED*, é possível substituir os valores em branco do atributo *CLEAN_TITLE* &nbsp;como **NO**.

In [38]:
df[df['clean_title'].isnull() | df['accident'].isnull()]

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
20,20,Land,Rover Defender S,2023,7978,Gasoline,5.2L V10 40V PDI DOHC,8-Speed Automatic,Santorini Black Metallic,Black,None reported,NaN,41998
45,45,RAM,1500 Laramie,2022,27352,Gasoline,5.7L V8 16V MPFI OHV,8-Speed Automatic,Diamond Black,Black,None reported,NaN,42599
51,51,Tesla,Model 3 Long Range,2020,38360,NaN,Dual Motor - Standard,Automatic,White,Black,None reported,NaN,59598
60,60,Mercedes-Benz,GLS 450 Base 4MATIC,2017,44147,Gasoline,3.0L V6 24V GDI DOHC Twin Turbo,9-Speed Automatic,White,Parchment.,At least 1 accident or damage reported,NaN,29999
70,70,McLaren,570S Spider,2023,3254,Gasoline,3.8L V8 32V MPFI DOHC Twin Turbo,7-Speed Automatic with Auto-Shift,Vega Blue,Black,None reported,NaN,92500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188507,188507,Chevrolet,Corvette Stingray w/3LT,2023,3053,Gasoline,490.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,NaN,NaN,61999
188511,188511,Land,Rover Range Rover Velar P380 SE R-Dynamic,2022,15198,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Firenze Red,Ebony,None reported,NaN,23739
188513,188513,Lexus,GS 350 F Sport,2022,82046,Gasoline,3.5L V6 24V PDI DOHC,8-Speed Automatic,Caviar,Black,None reported,NaN,48990
188520,188520,Mercedes-Benz,GLC 300 GLC 300,2022,18031,Gasoline,2.0 Liter Turbo,Automatic,White,Black,None reported,NaN,39998


<br>

#### Verificação de atributos repetidos

In [39]:
dup_feat = df.duplicated().sum()
print('Número de observações repetidas:', dup_feat)

Número de observações repetidas: 0
